In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from model import touchdownNetwork
from dataset import nfl_touchdown_data
from train import touchdown_model_trainer, TD_FREQ
import warnings

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
TRAIN = "../features/train"
VAL = "../features/validate"
MASTER_FNAME = "master.csv"

In [ ]:
train_nfl_touchdown_data = nfl_touchdown_data(TRAIN, MASTER_FNAME)
train_nfl_touchdown_data_loader = DataLoader(train_nfl_touchdown_data, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=20)

val_nfl_touchdown_data = nfl_touchdown_data(VAL, MASTER_FNAME)
val_nfl_touchdown_data_loader = DataLoader(val_nfl_touchdown_data, batch_size=128, shuffle=False, drop_last=False, num_workers=20)

In [ ]:
# hyperparameters
epochs = 50
sensitivity_weight = 0.15
lr = 0.001
betas=(0.99, 0.999)

activation = nn.LeakyReLU()
model = touchdownNetwork(activation).to(device)
optimizer = 'Adam'

seed = 90
torch.manual_seed(seed)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    model_trainer = touchdown_model_trainer(
        model,
        optimizer,
        epochs,
        TD_FREQ,
        train_nfl_touchdown_data_loader,
        val_nfl_touchdown_data_loader,
        sensitivity_weight=sensitivity_weight,
        lr=lr,
        betas=betas
    )

    model_trainer.save_hyperparameters()
    model_trainer.fit()